# 1、什么是Handler 消息传递机制
## 1.1 Handler的组成
* Handler: 发送消息和处理消息
* MessageQueue: 消息队列，被Looper管理
* Looper: 从MessageQueue中读取消息，按照消息分发机制，分配到目标Handler


## 1.2 Handler 的使用和代码实例
# 2、源码分析
## 2.1 Handler 机制源码分析
创建Handler
* 获取Looper，获取Looper的MessageQueue
* ThreadLocal 线程局部变量，起到线程隔离的作用，每个线程只能获取到自己的Looper。
* Looper.myLooper 是通过Looper 中的静态变量（sThreadLocal）来得到每一个Thread 的Looper。每个Thread 的Looper都是以Looper 的静态变量sThreadLocal作为key，保存在每个线程的ThreadLocalMap中的。
* 主线程:ActivityThread 的main方法已经自动执行了Looper.prepare方法
* 子线程:要先执行prepare才行

## 2.2 Handler 发送消息
### 2.2.1 post方法
post() 和 postDelayed() 
post 调用postDelayed() 方法
postDelayed() 方法调用sendMessageDelayed() 方法
getPostMessage() 方法是将Runnable 封装成Message

### 2.2.2 send方法
* sendMessage() ：发送带有数据的信息
* sendMessageDelayed() ：发送带有数据的信息，并设置延迟时间
* sendEmptyMessage() ：发送空消息
不管是post 还是send，最后都会调用sendMessageDelayed()->sendMessageAtTime() 方法。都是用SystemClock.uptimeMillis() 获取当前时间，加上设置的延迟时间，得到一个执行时间，然后调用sendMessageAtTime()->enqueueMessage() 做进一步逻辑。
ps：SystemClock.uptimeMillis() 获取的是自开机到当前时刻的毫秒数，和系统时间无关，不受系统时间的影响。不能用System.currentTimeMillis() 获取当前时间，因为系统时间可以被用户修改。

Java
'''
<!-- 代码简略版 -->
public class Handler {

    @UnsupportedAppUsage
    final Looper mLooper;
    final MessageQueue mQueue;

    public Handler(Callback callback, boolean async) {
        mLooper = Looper.myLooper();
        if (mLooper == null) {
            throw new RuntimeException("Can't create handler inside thread that has not called Looper.prepare()");
        }
        mQueue = mLooper.mQueue;
    }


    private boolean enqueueMessage(@NonNull MessageQueue queue, @NonNull Message msg,
            long uptimeMillis) {
        msg.target = this;
        msg.workSourceUid = ThreadLocalWorkSource.getUid();

        if (mAsynchronous) {
            msg.setAsynchronous(true);
        }
        return queue.enqueueMessage(msg, uptimeMillis);
    }

}
'''

## 2.3 Message 源码分析
* what: 用户自定义的消息识别标识
* arg1 和 arg2: 携带数据，只能传递int 类型数据
* obj: 可以传递object 类型数据，可以传递任意类型数据
* data: 可以使用bundle 传递数据
消息池是以链表的形式存储的，Message 的成员变量next 指向下一个Message，Message 的成员变量target 指向Handler。
使用Message.obtain() 方法从消息池中获取Message 对象，如果消息池中没有Message 对象，则创建新的Message 对象。避免了频繁创建和销毁Message 对象，提高了性能。

## 2.4 Looper 源码分析
* Looper.myLooper() 获取当前线程的Looper
* Looper.prepare() 初始化Looper实例
* Looper.prepareMainLooper() 主线程main方法中调用的方法，初始化Looper实例，并指定Looper 的MessageQueue 为系统的主消息队列
Looper中包括：
mQueue: 消息队列
mThread: 当前线程
MessageQueue 是在Looper()中创建的
loop()方法，死循环做三件事：
1. 调用next() 方法，轮询MessageQueue 中的消息
2. 通过dispatchMessage() 方法，将消息分发给目标 Handler
3. 通过recycleUnchecked() 方法回收Message

dispatchMessage() 方法：
* 优先级从高到低：
1. 如果Message 的callback 不为空，则通过handleCallback() 方法，调用callback 的run() 方法
2. 如果Message 的target 的mCallback 不为空，则通过mCallback 的handleMessage() 方法，处理消息
3. handlemessage(msg) 方法，重写handleMessage() 方法，处理消息

## 2.5 MessageQueue 源码分析
### 2.5.1 存放信息的方法：enqueueMessage()
* 将Message 存放在MessageQueue 中
* 如果MessageQueue 中没有Message 对象，作为队头的message
* 如果MessageQueue 中已经有Message 对象，则根据Message 的when 属性，将Message 插入到合适的位置

### 2.5.2 获取信息的方法：next()
* 阻塞操作，当nativePollOnce() 等待nextPollTimeoutMillis 时间后，如果MessageQueue 被唤醒，则返回Message 对象
* 如果获取的message 是null，或者msg 的target 为null，找下一个消息
* 如果消息不为空，根据when 属性，判断是否需要延迟处理，但是最后都会返回Message 对象
* 如果队列中没有消息，则阻塞在nativePollOnce() 方法
